In [ ]:
import numpy as np
import pandas as pd

In [ ]:
temp_df = pd.read_csv('/content/IMDB Dataset.csv')

In [ ]:
df = temp_df.iloc[:10000]

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df.drop_duplicates(inplace=True)

<ipython-input-12-16cdb8520be8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [ ]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [ ]:
df['review'] = df['review'].apply(remove_tags)

<ipython-input-14-00e93c2a3043>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(remove_tags)


In [ ]:
df['review'] = df['review'].apply(lambda x:x.lower())

<ipython-input-15-afca29351ee0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x:x.lower())


In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
<ipython-input-17-156e99996c0f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))


In [ ]:
df['review']

,review
0,one reviewers mentioned watching 1 oz episode ...
1,wonderful little production. filming technique...
2,thought wonderful way spend time hot summer we...
3,basically there's family little boy (jake) thi...
4,"petter mattei's ""love time money"" visually stu..."
...,...
9995,"fun, entertaining movie wwii german spy (julie..."
9996,"give break. anyone say ""good hockey movie""? kn..."
9997,movie bad movie. watching endless series bad h...
9998,"movie probably made entertain middle school, e..."


In [ ]:
import gensim
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [ ]:
import nltk
nltk.download('punkt_tab')

story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
model = gensim.models.Word2Vec(window=10,min_count=2,workers=4)

In [ ]:
model.build_vocab(story)

In [ ]:
model.train(story,total_examples=model.corpus_count,epochs=model.epochs)

(5875567, 6212140)

In [ ]:
len(model.wv.index_to_key)

31845

In [ ]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [ ]:
document_vector(df['review'].values[0])

array([-0.20092857,  0.4509839 ,  0.1503521 ,  0.22937334, -0.00620019,
       -0.66873133,  0.22271886,  0.94565886, -0.341813  , -0.24126117,
       -0.29990667, -0.4107857 ,  0.11371356,  0.1089434 ,  0.1301982 ,
       -0.24511687,  0.00361528, -0.4033442 , -0.08123921, -0.68430245,
        0.01500943,  0.23492743,  0.12460236, -0.28862146, -0.31923825,
       -0.01457746, -0.35116854,  0.02948632, -0.30861047,  0.0665437 ,
        0.38114348, -0.01603263,  0.24268918, -0.27048504, -0.10985756,
        0.46947774,  0.11813453, -0.36910698, -0.2493167 , -0.7140963 ,
        0.11580259, -0.2016665 , -0.0175035 , -0.07169503,  0.5193527 ,
       -0.11477934, -0.24852344,  0.01014947,  0.11922338,  0.39842442,
        0.09796438, -0.34481922, -0.32955506, -0.1140084 , -0.09905618,
        0.1522179 ,  0.2804382 ,  0.10771969, -0.31375238,  0.11654204,
        0.09357829,  0.14851815, -0.07697837, -0.05759284, -0.354494  ,
        0.18683615,  0.05103559,  0.17823146, -0.35562515,  0.26

In [ ]:
from tqdm import tqdm

X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))


100%|██████████| 9983/9983 [09:52<00:00, 16.85it/s]


In [ ]:
X = np.array(X)

In [ ]:
X[0]

array([-0.20092857,  0.4509839 ,  0.1503521 ,  0.22937334, -0.00620019,
       -0.66873133,  0.22271886,  0.94565886, -0.341813  , -0.24126117,
       -0.29990667, -0.4107857 ,  0.11371356,  0.1089434 ,  0.1301982 ,
       -0.24511687,  0.00361528, -0.4033442 , -0.08123921, -0.68430245,
        0.01500943,  0.23492743,  0.12460236, -0.28862146, -0.31923825,
       -0.01457746, -0.35116854,  0.02948632, -0.30861047,  0.0665437 ,
        0.38114348, -0.01603263,  0.24268918, -0.27048504, -0.10985756,
        0.46947774,  0.11813453, -0.36910698, -0.2493167 , -0.7140963 ,
        0.11580259, -0.2016665 , -0.0175035 , -0.07169503,  0.5193527 ,
       -0.11477934, -0.24852344,  0.01014947,  0.11922338,  0.39842442,
        0.09796438, -0.34481922, -0.32955506, -0.1140084 , -0.09905618,
        0.1522179 ,  0.2804382 ,  0.10771969, -0.31375238,  0.11654204,
        0.09357829,  0.14851815, -0.07697837, -0.05759284, -0.354494  ,
        0.18683615,  0.05103559,  0.17823146, -0.35562515,  0.26

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(df['sentiment'])

In [ ]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)